In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Embedding, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support

In [4]:
full_df = pd.read_csv('data/affcon_final.csv')

In [2]:
# Full data 
full_df = pd.read_csv('data/full_diplomacy_data.csv')
full_df = full_df.rename(columns={"text": "Input.full_text", "meta.deception_quadrant": "Input.deception_quadrant"})

In [3]:
full_df = full_df[full_df['meta.speaker_intention'] == 'Truth']

In [6]:
full_df.head()

,timestamp,Input.full_text,speaker,reply_to,conversation_id,meta.speaker_intention,meta.receiver_perception,meta.receiver,meta.absolute_message_index,meta.relative_message_index,meta.year,meta.game_score,meta.game_score_delta,Input.deception_quadrant
0,74,Germany!\n\nJust the person I want to speak wi...,italy-Game1,NaN,Game1-italy-germany,Truth,Truth,germany-Game1,74,0,1901,3,0,Straightforward
1,76,"You've whet my appetite, Italy. What's the sug...",germany-Game1,Game1-italy-germany-0,Game1-italy-germany,Truth,Truth,italy-Game1,76,1,1901,3,0,Straightforward
2,86,👍,italy-Game1,Game1-italy-germany-1,Game1-italy-germany,Truth,Truth,germany-Game1,86,2,1901,3,0,Straightforward
3,87,It seems like there are a lot of ways that cou...,germany-Game1,Game1-italy-germany-2,Game1-italy-germany,Truth,Truth,italy-Game1,87,3,1901,3,0,Straightforward
4,89,"Yeah, I can’t say I’ve tried it and it works, ...",italy-Game1,Game1-italy-germany-3,Game1-italy-germany,Truth,NaN,germany-Game1,89,4,1901,3,0,Unknown


In [4]:
full_df = full_df.dropna() # dataset contains NaN values, dropping NaNs here

X = full_df['Input.full_text']

full_df["Input.deception_quadrant"] = full_df["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y = full_df['Input.deception_quadrant']

le = LabelEncoder() # this can convert our categories into labels, make sure you don't have NaNs or Nulls in your data first
y = le.fit_transform(y)
print(y.shape)

# we reshape 
y = y.reshape(-1,1) # the -1 allows it to have whatever number went in there
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(14775,)
(14775, 1)
(11820,)
(11820, 1)
(2955,)
(2955, 1)


In [5]:
max_words = 1000
max_len = 100

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

sequences = tok.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences,maxlen=max_len)
# X_train = sequence.pad_sequences(sequences)

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


## LSTM Model

In [7]:
Inp = Input(name='inputs',shape=[max_len])
x = Embedding(max_words,50,input_length=max_len)(Inp)
x = LSTM(64,name='LSTM_01')(x)
x = Dropout(0.5,name='Dropout')(x)
x = Dense(128,activation='relu',name='Dense_01')(x)
# x = Dropout(0.5,name='Dropout')(x)
out = Dense(1,activation='sigmoid', name='output')(x)

model = Model(inputs=Inp,outputs=out)

model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           50000     
_________________________________________________________________
LSTM_01 (LSTM)               (None, 64)                29440     
_________________________________________________________________
Dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
Dense_01 (Dense)             (None, 128)               8320      
_________________________________________________________________
output (Dense)               (None, 1)                 129       
Total params: 87,889
Trainable params: 87,889
Non-trainable params: 0
__________________________________________________

In [8]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

model.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2,
          callbacks=[early_stop])

Epoch 1/15
74/74 [==============================] - 2s 26ms/step - loss: 0.2102 - acc: 0.9474 - f1_m: 0.9705 - precision_m: 0.9566 - recall_m: 0.9905 - val_loss: 0.1742 - val_acc: 0.9585 - val_f1_m: 0.9791 - val_precision_m: 0.9592 - val_recall_m: 1.0000
Epoch 2/15
74/74 [==============================] - 1s 19ms/step - loss: 0.1824 - acc: 0.9567 - f1_m: 0.9778 - precision_m: 0.9568 - recall_m: 1.0000 - val_loss: 0.1712 - val_acc: 0.9585 - val_f1_m: 0.9791 - val_precision_m: 0.9592 - val_recall_m: 1.0000
Epoch 3/15
74/74 [==============================] - 1s 19ms/step - loss: 0.1743 - acc: 0.9567 - f1_m: 0.9778 - precision_m: 0.9567 - recall_m: 1.0000 - val_loss: 0.1766 - val_acc: 0.9585 - val_f1_m: 0.9791 - val_precision_m: 0.9592 - val_recall_m: 1.0000


In [9]:
test_sequences_LSTM = tok.texts_to_sequences(X_test)
X_test_LSTM = sequence.pad_sequences(test_sequences_LSTM,maxlen=max_len)
model.evaluate(X_test_LSTM,y_test)

93/93 [==============================] - 1s 7ms/step - loss: 0.2039 - acc: 0.9486 - f1_m: 0.9734 - precision_m: 0.9489 - recall_m: 1.0000


[0.20389054715633392,
 0.9485617876052856,
 0.9733641147613525,
 0.948924720287323,
 1.0]

In [10]:
a = model.predict(X_test_LSTM).round()
precision_recall_fscore_support(y_test, a, average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4742808798646362, 0.5, 0.4868009725599166, None)

## CNN Model

In [8]:
model_CNN = Sequential(name="CNN_with_embeddings")
model_CNN.add(Embedding(max_words, 50, input_length=max_len))
model_CNN.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model_CNN.add(MaxPooling1D(pool_size=2))
model_CNN.add(Flatten())
model_CNN.add(Dropout(0.5))
model_CNN.add(Dense(10, activation='relu'))
model_CNN.add(Dense(1, activation='sigmoid'))

model_CNN.compile(loss='binary_crossentropy', 
              optimizer= 'adam',
              metrics=['acc',f1_m,precision_m, recall_m])

model_CNN.summary()

Model: "CNN_with_embeddings"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 93, 32)            12832     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 46, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1472)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1472)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                14730     
_________________________________________________________________
dense_3 (Dense)              (None, 1)         

In [9]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

model_CNN.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2,
          callbacks=[early_stop])

Epoch 1/15
74/74 [==============================] - 1s 15ms/step - loss: 0.2746 - acc: 0.9355 - f1_m: 0.9590 - precision_m: 0.9543 - recall_m: 0.9794 - val_loss: 0.1717 - val_acc: 0.9590 - val_f1_m: 0.9784 - val_precision_m: 0.9578 - val_recall_m: 1.0000
Epoch 2/15
74/74 [==============================] - 1s 10ms/step - loss: 0.1877 - acc: 0.9532 - f1_m: 0.9760 - precision_m: 0.9532 - recall_m: 1.0000 - val_loss: 0.1713 - val_acc: 0.9590 - val_f1_m: 0.9784 - val_precision_m: 0.9578 - val_recall_m: 1.0000
Epoch 3/15
74/74 [==============================] - 1s 11ms/step - loss: 0.1836 - acc: 0.9532 - f1_m: 0.9759 - precision_m: 0.9532 - recall_m: 1.0000 - val_loss: 0.1718 - val_acc: 0.9590 - val_f1_m: 0.9784 - val_precision_m: 0.9578 - val_recall_m: 1.0000


In [14]:
test_sequences_CNN = tok.texts_to_sequences(X_test)
X_test_CNN = sequence.pad_sequences(test_sequences_CNN,maxlen=max_len)

model_CNN.evaluate(X_test_CNN,y_test)

93/93 [==============================] - 0s 3ms/step - loss: 0.1707 - acc: 0.9597 - f1_m: 0.9790 - precision_m: 0.9594 - recall_m: 1.0000


[0.17066460847854614,
 0.9597292542457581,
 0.9789561033248901,
 0.9593719244003296,
 1.0]

In [16]:
a_cnn = model_CNN.predict(X_test_CNN).round()
precision_recall_fscore_support(y_test, a_cnn, average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4798646362098139, 0.5, 0.48972543602141255, None)